### Spark Streaming (Kafka)

In [1]:
!pip install pyspark==3.4.1


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
import pyspark
print(pyspark.__version__)

3.4.1


In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, window, sum, avg, count, from_json, from_unixtime
from pyspark.sql.types import StructType, StringType, DoubleType, LongType
import os

JAR_PATH = os.path.abspath("jars/spark-sql-kafka-0-10_2.13-3.4.1.jar")

# Проверка наличия файла
if not os.path.exists(JAR_PATH):
    raise FileNotFoundError(f"JDBC драйвер не найден: {JAR_PATH}")

print(f"Найден JDBC драйвер: {JAR_PATH}")

Найден JDBC драйвер: /opt/workspace/jars/spark-sql-kafka-0-10_2.13-3.4.1.jar


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, window, sum, from_json, from_unixtime
from pyspark.sql.types import StructType, StringType, DoubleType, LongType
import os

# Создание SparkSession
spark = SparkSession.builder \
    .appName("Kafka Happy Box Revenue") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.4.1") \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

# Схема JSON
schema = StructType()\
    .add("product", StringType())\
    .add("amount", DoubleType())\
    .add("timestamp", LongType())

# Kafka
df_raw = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "...:...") \
    .option("subscribe", "sales") \
    .option("startingOffsets", "latest") \
    .load()

# JSON -> Struct
df_json = df_raw.selectExpr("CAST(value AS STRING) as json_data", "timestamp") \
    .select(from_json(col("json_data"), schema).alias("data"))

# Извлекаем поля
df_parsed = df_json.select(
    col("data.product").alias("product_name"),
    col("data.amount"),
    from_unixtime(col("data.timestamp")).cast("timestamp").alias("event_time")
)

# Фильтрация
df_filtered = df_parsed.filter(col("product_name") == "Happy Box 30")

df_filtered = df_filtered.withWatermark("event_time", "30 seconds")

# Оконная агрегация
df_agg = df_filtered.groupBy(
    window(col("event_time"), "10 seconds")
).agg(
    sum("amount").alias("total_revenue")
).select(
    col("window.start").alias("window_start"),
    col("window.end").alias("window_end"),
    col("total_revenue")
)

query = df_agg.writeStream \
    .outputMode("complete") \
    .format("console") \
    .option("truncate", False) \
    .start()

query.awaitTermination()


/usr/local/lib/python3.9/dist-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found


:: loading settings :: url = jar:file:/usr/local/lib/python3.9/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d04fb8de-63c1-4a6e-8904-fe0825e0a9df;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.4.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.4.1 in central
	found org.apache.kafka#kafka-clients;3.3.2 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.1 in central
	found org.slf4j#slf4j-api;2.0.6 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
:: resolution report :: resolve 322ms :: artifacts dl 10ms
	:: modules in us

-------------------------------------------
Batch: 0
-------------------------------------------
+------------+----------+-------------+
|window_start|window_end|total_revenue|
+------------+----------+-------------+
+------------+----------+-------------+



-------------------------------------------
Batch: 1
-------------------------------------------
+-------------------+-------------------+-------------+
|window_start       |window_end         |total_revenue|
+-------------------+-------------------+-------------+
|2025-07-31 15:59:20|2025-07-31 15:59:30|703.81       |
|2025-07-31 15:59:30|2025-07-31 15:59:40|398.2        |
+-------------------+-------------------+-------------+



...

-------------------------------------------
Batch: 1537
-------------------------------------------
+-------------------+-------------------+------------------+
|window_start       |window_end         |total_revenue     |
+-------------------+-------------------+------------------+
|2025-07-31 16:01:10|2025-07-31 16:01:20|6291.579999999999 |
|2025-07-31 16:55:30|2025-07-31 16:55:40|3543.36           |
|2025-07-31 16:56:30|2025-07-31 16:56:40|7871.849999999999 |
|2025-07-31 17:00:30|2025-07-31 17:00:40|6894.51           |
|2025-07-31 16:12:50|2025-07-31 16:13:00|6365.38           |
|2025-07-31 16:45:50|2025-07-31 16:46:00|7427.139999999999 |
|2025-07-31 16:49:40|2025-07-31 16:49:50|3699.95           |
|2025-07-31 16:10:00|2025-07-31 16:10:10|6055.72           |
|2025-07-31 16:23:50|2025-07-31 16:24:00|9774.24           |
|2025-07-31 16:36:10|2025-07-31 16:36:20|5665.530000000001 |
|2025-07-31 16:57:50|2025-07-31 16:58:00|6268.7699999999995|
|2025-07-31 17:00:10|2025-07-31 17:00:20|4821.

-------------------------------------------
Batch: 1538
-------------------------------------------
+-------------------+-------------------+------------------+
|window_start       |window_end         |total_revenue     |
+-------------------+-------------------+------------------+
|2025-07-31 16:01:10|2025-07-31 16:01:20|6291.579999999999 |
|2025-07-31 16:55:30|2025-07-31 16:55:40|3543.36           |
|2025-07-31 16:56:30|2025-07-31 16:56:40|7871.849999999999 |
|2025-07-31 17:00:30|2025-07-31 17:00:40|6894.51           |
|2025-07-31 16:12:50|2025-07-31 16:13:00|6365.38           |
|2025-07-31 16:45:50|2025-07-31 16:46:00|7427.139999999999 |
|2025-07-31 16:49:40|2025-07-31 16:49:50|3699.95           |
|2025-07-31 16:10:00|2025-07-31 16:10:10|6055.72           |
|2025-07-31 16:23:50|2025-07-31 16:24:00|9774.24           |
|2025-07-31 16:36:10|2025-07-31 16:36:20|5665.530000000001 |
|2025-07-31 16:57:50|2025-07-31 16:58:00|6268.7699999999995|
|2025-07-31 17:00:10|2025-07-31 17:00:20|4821.

-------------------------------------------
Batch: 1539
-------------------------------------------
+-------------------+-------------------+------------------+
|window_start       |window_end         |total_revenue     |
+-------------------+-------------------+------------------+
|2025-07-31 16:01:10|2025-07-31 16:01:20|6291.579999999999 |
|2025-07-31 16:55:30|2025-07-31 16:55:40|3543.36           |
|2025-07-31 16:56:30|2025-07-31 16:56:40|7871.849999999999 |
|2025-07-31 17:00:30|2025-07-31 17:00:40|6894.51           |
|2025-07-31 16:12:50|2025-07-31 16:13:00|6365.38           |
|2025-07-31 16:45:50|2025-07-31 16:46:00|7427.139999999999 |
|2025-07-31 16:49:40|2025-07-31 16:49:50|3699.95           |
|2025-07-31 16:10:00|2025-07-31 16:10:10|6055.72           |
|2025-07-31 16:23:50|2025-07-31 16:24:00|9774.24           |
|2025-07-31 16:36:10|2025-07-31 16:36:20|5665.530000000001 |
|2025-07-31 16:57:50|2025-07-31 16:58:00|6268.7699999999995|
|2025-07-31 17:00:10|2025-07-31 17:00:20|4821.

-------------------------------------------
Batch: 1540
-------------------------------------------
+-------------------+-------------------+------------------+
|window_start       |window_end         |total_revenue     |
+-------------------+-------------------+------------------+
|2025-07-31 16:01:10|2025-07-31 16:01:20|6291.579999999999 |
|2025-07-31 16:55:30|2025-07-31 16:55:40|3543.36           |
|2025-07-31 16:56:30|2025-07-31 16:56:40|7871.849999999999 |
|2025-07-31 17:00:30|2025-07-31 17:00:40|6894.51           |
|2025-07-31 16:12:50|2025-07-31 16:13:00|6365.38           |
|2025-07-31 16:45:50|2025-07-31 16:46:00|7427.139999999999 |
|2025-07-31 16:49:40|2025-07-31 16:49:50|3699.95           |
|2025-07-31 16:10:00|2025-07-31 16:10:10|6055.72           |
|2025-07-31 16:23:50|2025-07-31 16:24:00|9774.24           |
|2025-07-31 16:36:10|2025-07-31 16:36:20|5665.530000000001 |
|2025-07-31 16:57:50|2025-07-31 16:58:00|6268.7699999999995|
|2025-07-31 17:00:10|2025-07-31 17:00:20|4821.

-------------------------------------------
Batch: 1541
-------------------------------------------
+-------------------+-------------------+------------------+
|window_start       |window_end         |total_revenue     |
+-------------------+-------------------+------------------+
|2025-07-31 16:01:10|2025-07-31 16:01:20|6291.579999999999 |
|2025-07-31 16:55:30|2025-07-31 16:55:40|3543.36           |
|2025-07-31 16:56:30|2025-07-31 16:56:40|7871.849999999999 |
|2025-07-31 17:00:30|2025-07-31 17:00:40|6894.51           |
|2025-07-31 16:12:50|2025-07-31 16:13:00|6365.38           |
|2025-07-31 16:45:50|2025-07-31 16:46:00|7427.139999999999 |
|2025-07-31 16:49:40|2025-07-31 16:49:50|3699.95           |
|2025-07-31 16:10:00|2025-07-31 16:10:10|6055.72           |
|2025-07-31 16:23:50|2025-07-31 16:24:00|9774.24           |
|2025-07-31 16:36:10|2025-07-31 16:36:20|5665.530000000001 |
|2025-07-31 16:57:50|2025-07-31 16:58:00|6268.7699999999995|
|2025-07-31 17:00:10|2025-07-31 17:00:20|4821.

In [ ]:
# from pyspark.sql import SparkSession
# from pyspark.sql.functions import col, window, count

# spark = SparkSession.builder \
#     .appName("Kafka Number Counter") \
#     .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.4.1") \
#     .getOrCreate()

# spark.sparkContext.setLogLevel("ERROR")

# # опционально — узнать, какую scala-версию в действительности использует runtime:
# print("Scala version:", spark._jvm.scala.util.Properties.versionString())

# df_raw = (spark.readStream
#     .format("kafka")
#     .option("kafka.bootstrap.servers", "de-db.groundplace.ru:9092")
#     .option("subscribe", "numbers")
#     .option("startingOffsets", "latest")
#     .load())

# df_numbers = (df_raw
#     .selectExpr("CAST(value AS STRING) as value", "timestamp")
#     .withColumn("number", col("value").cast("int")))

# aggregated = (df_numbers
#     .groupBy(window(col("timestamp"), "5 seconds"))
#     .agg(count("number").alias("count")))

# query = (aggregated.writeStream
#     .outputMode("complete")
#     .format("console")
#     .option("truncate", False)
#     .start())

# query.awaitTermination(20)


In [ ]:
!ls jars